## Himmelblau Graphics

This notebook shows the code to produce the Himmelblau graphics in the getting_started.ipynb notebook.

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from IPython.display import clear_output
from PIL import Image

import bofire.strategies.api as strategies
from bofire.data_models.acquisition_functions.api import qLogEI
from bofire.data_models.domain.api import Domain, Inputs, Outputs
from bofire.data_models.features.api import ContinuousInput, ContinuousOutput
from bofire.data_models.objectives.api import MinimizeObjective
from bofire.data_models.strategies.api import SoboStrategy


plotly_config = {
    'toImageButtonOptions': {
        'format': 'png',  # one of png, svg, jpeg, webp
        'filename': 'custom_image',
        'scale': 4  # Multiply title/legend/axis/canvas sizes by this factor
    }
}


c:\Users\dobresse.AGILENT\.conda\envs\bofire\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Explore Himmelblau Function

In [2]:
def himmelblau(x_1, x_2):
    return ((x_1**2 + x_2 - 11)**2+(x_1 + x_2**2 -7)**2)

The function has 4 minima

In [3]:
x_1_vec = np.linspace(-6, 6, 100)
x_2_vec = np.linspace(-6, 6, 100)

x_1, x_2 = np.meshgrid(x_1_vec, x_2_vec)
results = himmelblau(x_1, x_2)

x_minima = [
        [3.0, 2.0],
        [-2.805118, 3.131312],
        [-3.779310, -3.283186],
        [3.584428, -1.848126],
    ]
y_minima = np.zeros(4)


#### Contour plot

In [ ]:
fig = go.Figure(data=go.Contour(z=results, x=x_1_vec, y=x_2_vec, contours_coloring="heatmap", zmin=0, zmax=600))
fig.update_layout(
    title='Himmelblau Function',
    autosize=False,
    width=750,
    height=750,
    margin={"l": 65, "r": 50, "b": 65, "t": 90},
    scene={
        "xaxis_title": 'x_1',
        "yaxis_title": 'x_2',
        "zaxis_title": 'y',
    })
for i in range(4):
    fig.add_trace(go.Scatter(x=[x_minima[i][0]], y=[x_minima[i][1]],
                             mode='markers', showlegend=False,
                             marker={"size": 10, "color": 'red'}))
fig.show(config=plotly_config)

#### 3D plot

In [17]:
fig = go.Figure(data=go.Surface(z=results, x=x_1_vec, y=x_2_vec))
fig.update_layout(
    title='Himmelblau Function',
    autosize=False,
    width=750,
    height=750,
    margin={"l": 65, "r": 50, "b": 65, "t": 90},
    scene={
        "xaxis_title": 'x_1',
        "yaxis_title": 'x_2',
        "zaxis_title": 'y',
    })
for i in range(4):
    fig.add_trace(go.Scatter3d(x=[x_minima[i][0]], y=[x_minima[i][1]], z=[y_minima[i]],
                             mode='markers', showlegend=False,
                             marker={"size": 10, "color": 'red'}))
fig.show()

#### Optimization Outputs

In [6]:
objective = MinimizeObjective()
output_feature = ContinuousOutput(key="y", objective=objective)

#### Optimization Inputs


In [7]:
input_feature_1 = ContinuousInput(key="x1", bounds=(-6, 6))
input_feature_2 = ContinuousInput(key="x2", bounds=(-6, 6))

#### Optimization Domain

In [8]:
domain = Domain(
    inputs=Inputs(features=[input_feature_1, input_feature_2]),
    outputs=Outputs(features=[output_feature]),
)

#### Draw candidates and execute experiments

In [18]:
candidates = domain.inputs.sample(10, seed=19)

experimental_output = candidates.apply(
    lambda row: himmelblau(row["x1"], row["x2"]), axis=1
)

experiments = candidates.copy()
experiments["y"] = experimental_output


#### Defining an optimization strategy

In [19]:
sobo_strategy_data_model = SoboStrategy(
    domain=domain, acquisition_function=qLogEI(), seed=19
)

sobo_strategy = strategies.map(sobo_strategy_data_model)

#### Define function to plot model mean prediction

In [20]:
def plot_himmelblau_opt(samples, model_prediction, iteration):
    fig = go.Figure(data=go.Contour(z=model_prediction, x=x_1_vec, y=x_2_vec, contours_coloring="heatmap", colorbar={"title": 'y'}, zmin=0, zmax=600))
    fig.update_layout(
        title=f'Optimization iteration: {iteration}',
        autosize=False,
        width=400,
        height=350,
        margin={"l": 10, "r": 10, "b": 10, "t": 40},
        xaxis_title = 'x1',
        yaxis_title = 'x2',
        legend_title = 'y',
        yaxis={'range': [-6, 6]},
        xaxis={'range': [-6, 6]}
        )

    for i in range(4):
       fig.add_trace(go.Scatter(x=[x_minima[i][0]], y=[x_minima[i][1]],
                               mode='markers', showlegend=False,
                               marker={"size": 10, "color": 'red'}))
    fig.add_trace(go.Scatter(x=samples["x1"], y=samples["x2"], mode='lines', showlegend=False, line={"color": 'blue', "width": 1}))
    fig.add_trace(go.Scatter(
        x=samples["x1"],
        y=samples["x2"],
        mode='markers',
        showlegend=False,
        marker={
            "size": 5,
            "color": 'white',
            "symbol": 'circle-open'}))
    return fig

#### Run the optimization loop

In [22]:
sobo_strategy.tell(experiments=experiments)
prediction = sobo_strategy.predict(pd.DataFrame({"x1": x_1.flatten(), "x2": x_2.flatten()}))
pred = np.array(prediction["y_pred"]).reshape(100, 100)


for i in range(30):
    new_candidates = sobo_strategy.ask(candidate_count=1)

    new_experiments = new_candidates.copy()
    new_experiments["y"] = new_candidates.apply(
        lambda row: himmelblau(row["x1"], row["x2"]), axis=1
    )
    experiments = pd.concat([experiments, new_experiments], join="inner").reset_index(
        drop=True
    )

    sobo_strategy.tell(experiments=experiments)

    clear_output(wait=True)

    # Plot the optimization process
    prediction = sobo_strategy.predict(pd.DataFrame({"x1": x_1.flatten(), "x2": x_2.flatten()}))
    pred = np.array(prediction["y_pred"]).reshape(100, 100)
    fig = plot_himmelblau_opt(experiments, pred, i)
    fig.show(config=plotly_config)

    fig.write_image(f"himmelblau_{i}.png", engine="orca")

#### Generate Himmelblau GIF

In [58]:
# Liste zum Speichern der Bilder erstellen
images = []

# Jedes Bild laden und zur Liste hinzufügen
for i in range(30):
    try:
        img = Image.open(f"himmelblau_{i}.png")
        images.append(img)
    except FileNotFoundError:
        print(f"Datei himmelblau_{i}.png nicht gefunden. Überspringe diese Datei.")

# Überprüfen, ob Bilder zum Speichern vorhanden sind
if images:
    # Bilder als GIF speichern
    images[0].save("himmelblau_optimization.gif", save_all=True, append_images=images[1:], duration=600, loop=0)
    print("GIF erfolgreich als himmelblau.gif erstellt.")
else:
    print("Keine Bilder gefunden, um ein GIF zu erstellen.")

GIF erfolgreich als himmelblau.gif erstellt.
